5. Testing areas

In [6]:
from bs4 import BeautifulSoup
import pandas as pd
import datetime 
from decimal import Decimal

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [8]:
# Tạo bảng
columns = ['MMYYYY',
            'Bank_name',
            'Bank_code', 
            'Peer_No',
            'Volume_type',
            'Segment',
            'Off_1M', 'Off_2M', 'Off_3M', 'Off_6M','Off_12M', 'Off_13M', 'Off_18M', 'Off_24M', 'Off_36M',
            'Onl_1M', 'Onl_2M', 'Onl_3M', 'Onl_6M','Onl_12M', 'Onl_13M', 'Onl_18M','Onl_24M','Onl_36M'
            ]


Update_inst = pd.DataFrame(columns=columns)

url_chrome = r"D:\Chromedriver\chromedriver.exe"


In [9]:
Update_inst

,MMYYYY,Bank_name,Bank_code,Peer_No,Volume_type,Segment,Off_1M,Off_2M,Off_3M,Off_6M,...,Off_36M,Onl_1M,Onl_2M,Onl_3M,Onl_6M,Onl_12M,Onl_13M,Onl_18M,Onl_24M,Onl_36M


In [10]:
def convert_to_float(value):
    return round(Decimal(float(value.replace(',', '.').replace('%', '').replace('*', ''))),3)


1. ACB

In [11]:
url_ACB = 'https://www.acb.com.vn/lai-suat-tien-gui' # Done

# Send an HTTP GET request to the URL and retrieve the content
chrome_options = Options()
driver = webdriver.Chrome(service=Service(executable_path=url_chrome), options=chrome_options)
driver.maximize_window()
acb = driver.get(url_ACB)

driver.implicitly_wait(20) # driver timeout to stop forever loading
ACB_html_content = driver.page_source #get page html content
ACB_soup = BeautifulSoup(ACB_html_content, 'html.parser') # Parse the HTML content

offline_values_1M =convert_to_float(ACB_soup.find_all("tbody")[1]\
                                   .find_all("tr")[6]\
                                   .find_all("td", class_="xl68")[0]\
                                   .text.strip())
offline_values = [convert_to_float(ACB_soup.find_all("tbody")[1]\
                                   .find_all("tr")[x]\
                                   .find_all("td")[1]\
                                   .text.strip()) for x in [6, 7, 10, 12, 13, 15, 16, 17]]
online_values_200 = [convert_to_float(ACB_soup.find_all("tbody")[3]\
                                  .find_all("tr")[2]\
                                  .find_all("td")[y]\
                                  .text.strip()) for y in [2, 3, 4, 5, 6, 7, 7, 7, 7]]
online_values_Up200 = [convert_to_float(ACB_soup.find_all("tbody")[3]\
                                  .find_all("tr")[3]\
                                  .find_all("td")[y]\
                                  .text.strip()) for y in [1, 2, 3, 4, 5, 6, 6, 6, 6]]
online_values_Up1000 = [convert_to_float(ACB_soup.find_all("tbody")[3]\
                                  .find_all("tr")[4]\
                                  .find_all("td")[y]\
                                  .text.strip()) for y in [1, 2, 3, 4, 5, 6, 6, 6, 6]]
online_values_Up5000 = [convert_to_float(ACB_soup.find_all("tbody")[3]\
                                  .find_all("tr")[5]\
                                  .find_all("td")[y]\
                                  .text.strip()) for y in [1, 2, 3, 4, 5, 6, 6, 6, 6]]
surplus_values = [convert_to_float(ACB_soup.find_all("tbody")[2]\
                                  .find_all("tr")[1]\
                                  .find_all("td")[x]\
                                  .text.strip()) for x in [1,2,3]]

# //*[@id="block-id-4"]/div/div/div/div/table[1]/tbody/tr[4]/td[2]/span # 1M - 200-1000
# //*[@id="block-id-4"]/div/div/div/div/table[1]/tbody/tr[4]/td[3]/span # 2M - 200-1000
# //*[@id="block-id-4"]/div/div/div/div/table[1]/tbody/tr[3]/td[3]/span
online_values_Up200

[Decimal('3.400'),
 Decimal('3.500'),
 Decimal('3.600'),
 Decimal('4.700'),
 Decimal('4.750'),
 Decimal('4.800'),
 Decimal('4.800'),
 Decimal('4.800'),
 Decimal('4.800')]

In [12]:
# 'Bank_name', 

Update_inst.loc[1, ['MMYYYY',
                    'Bank_code', 
                    'Volume_type',
                    "Off_1M","Off_2M","Off_3M", "Off_6M", "Off_12M","Off_13M","Off_18M", "Off_24M", "Off_36M",
                    "Onl_1M","Onl_2M","Onl_3M","Onl_6M", "Onl_12M","Onl_13M","Onl_18M", "Onl_24M", "Onl_36M"]]\
                 = [datetime.datetime.now().strftime("%d/%m/%Y"),
                    'ACB',
                    '<200M',
                    offline_values_1M,
                    *offline_values,
                    *online_values_200
                    ]
Update_inst.loc[2, ['MMYYYY',
                    'Bank_code', 
                    'Volume_type',
                    "Off_1M","Off_2M","Off_3M", "Off_6M", "Off_12M","Off_13M","Off_18M", "Off_24M", "Off_36M",
                    "Onl_1M","Onl_2M","Onl_3M","Onl_6M", "Onl_12M","Onl_13M","Onl_18M", "Onl_24M", "Onl_36M"]]\
                 = [datetime.datetime.now().strftime("%d/%m/%Y"),
                    'ACB',
                    '200M-1B',
                    offline_values_1M + surplus_values[1],
                    *[x + surplus_values[1] for x in offline_values],
                     *online_values_Up200
                    ]
Update_inst.loc[3, ['MMYYYY',
                    'Bank_code', 
                    'Volume_type',
                    "Off_1M","Off_2M","Off_3M", "Off_6M", "Off_12M","Off_13M","Off_18M", "Off_24M", "Off_36M",
                    "Onl_1M","Onl_2M","Onl_3M","Onl_6M", "Onl_12M","Onl_13M","Onl_18M", "Onl_24M", "Onl_36M"]]\
                 = [datetime.datetime.now().strftime("%d/%m/%Y"),
                    'ACB',
                    '1B-5B',
                    offline_values_1M + surplus_values[1],
                    *[x + surplus_values[1] for x in offline_values],
                    *online_values_Up1000
                    ]
Update_inst.loc[4, ['MMYYYY',
                    'Bank_code', 
                    'Volume_type',
                    "Off_1M","Off_2M","Off_3M", "Off_6M", "Off_12M","Off_13M","Off_18M", "Off_24M", "Off_36M",
                    "Onl_1M","Onl_2M","Onl_3M","Onl_6M", "Onl_12M","Onl_13M","Onl_18M", "Onl_24M", "Onl_36M"]]\
                 = [datetime.datetime.now().strftime("%d/%m/%Y"),
                    'ACB',
                    '>5B',
                    offline_values_1M + surplus_values[1],
                    *[x + surplus_values[1] for x in offline_values],
                    *online_values_Up5000
                    ]


In [51]:
Update_inst

,MMYYYY,Bank_name,Bank_code,Peer_No,Volume_type,Segment,Off_1M,Off_2M,Off_3M,Off_6M,...,Off_36M,Onl_1M,Onl_2M,Onl_3M,Onl_6M,Onl_12M,Onl_13M,Onl_18M,Onl_24M,Onl_36M
1,14/11/2023,NaN,ACB,NaN,<200M,NaN,3.300,3.300,3.400,4.500,...,4.600,3.300,3.400,3.500,4.600,4.650,4.700,4.700,4.700,4.700
2,14/11/2023,NaN,ACB,NaN,200M-1B,NaN,3.450,3.450,3.550,4.650,...,4.750,3.400,3.500,3.600,4.700,4.750,4.800,4.800,4.800,4.800
3,14/11/2023,NaN,ACB,NaN,1B-5B,NaN,3.450,3.450,3.550,4.650,...,4.750,3.450,3.550,3.650,4.750,4.800,4.850,4.850,4.850,4.850
4,14/11/2023,NaN,ACB,NaN,>5B,NaN,3.450,3.450,3.550,4.650,...,4.750,3.500,3.600,3.700,4.800,4.850,4.900,4.900,4.900,4.900


2. MB Bank

In [41]:
chrome_options = Options()
driver = webdriver.Chrome(service=Service(executable_path=url_chrome), options=chrome_options)
driver.maximize_window()

MB = driver.get('https://www.mbbank.com.vn/Fee')
MB_Soup = BeautifulSoup(driver.page_source, 'html').find('div', class_= "toggle-content")
MB_list = [convert_to_float(MB_Soup\
                    .find_all('table')[0]\
                    .find_all('tr')[x]\
                    .find_all('td')[1]\
                    .text.strip()) for x in [5, 6, 7, 10, 16, 17, 19, 20, 21]]


In [38]:
# 'Bank_name', 
Update_inst.loc[1, ['MMYYYY',
                  'Bank_code', 
                  'Volume_type',
                  'Off_1M', 'Off_2M', 'Off_3M', 'Off_6M','Off_12M', 'Off_13M', 'Off_18M', 'Off_24M', 'Off_36M',
                  'Onl_1M', 'Onl_2M', 'Onl_3M', 'Onl_6M','Onl_12M', 'Onl_13M', 'Onl_18M','Onl_24M','Onl_36M']]\
                 = [datetime.datetime.now().strftime("%d/%m/%Y"),
                    'MB',
                    'All',
                    *MB_list,
                    *MB_list,
                    ]

3. TP bank

In [44]:
chrome_options = Options()
driver = webdriver.Chrome(service=Service(executable_path=url_chrome), options=chrome_options)
driver.maximize_window()
TPB = driver.get('https://tpb.vn/cong-cu-tinh-toan/lai-suat')
TPB_soup = BeautifulSoup(driver.page_source, 'html').find('table')

TPB_table = TPB_soup.find_all('table')[0]
TPB_table
# for x in range [0,6]:
#    the_list = TPB_table.find_all("tbody").find_all("tr")[x].find_all("td").text.strip()
# the_list
# value_offline_truong_an = [convert_to_float(TPB_table.find_all("tbody")[0]\
#                                   .find_all("tr")[y]\
#                                   .find_all("td")[1]\
#                                   .text.strip()) for y in [2, 3, 4, 5, 6, 7, 7, 7, 7]]
# value_online             = [convert_to_float(TPB_table.find_all("tbody")[0]\
#                                   .find_all("tr")[y]\
#                                   .find_all("td")[1]\
#                                   .text.strip()) for y in [2, 3, 4, 5, 6, 7, 7, 7, 7]]
# value_offline_cuoi_ky   = [convert_to_float(TPB_table.find_all("tbody")[0]\
#                                   .find_all("tr")[2]\
#                                   .find_all("td")[y]\
#                                   .text.strip()) for y in [2, 3, 4, 5, 6, 7, 7, 7, 7]]

# Update_inst.loc[1, ['MMYYYY',
#                     'Bank_code', 
#                     'Volume_type',
#                     "Off_1M","Off_2M","Off_3M", "Off_6M", "Off_12M","Off_13M","Off_18M", "Off_24M", "Off_36M",
#                     "Onl_1M","Onl_2M","Onl_3M","Onl_6M", "Onl_12M","Onl_13M","Onl_18M", "Onl_24M", "Onl_36M"]]\
#                  = [datetime.datetime.now().strftime("%d/%m/%Y"),
#                     'TPB',
#                     '<200M',
#                     offline_values_1M,
#                     *offline_values,
#                     *online_values_200
#                     ]

# //*[@id="tab-1"]/table[1]/tbody/tr[1]/td[2] # off 1M An loc
# //*[@id="tab-1"]/table[1]/tbody/tr[2]/td[2] # off 3M An loc
# //*[@id="tab-1"]/table[1]/tbody/tr[1]/td[4] # off 1M Lai cuoi ky

# //*[@id="tab-1"]/table[1]/tbody/tr[1]/td[3] # onl 1M
# //*[@id="tab-1"]/table[1]/tbody/tr[2]/td[3] # onl 3M


MaxRetryError: HTTPConnectionPool(host='localhost', port=59432): Max retries exceeded with url: /session/5bf51b6e4fbce05a8c89792616c12728/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001AD2FFC4410>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

4. Lienviet Post Bank

In [35]:
chrome_options = Options()
driver = webdriver.Chrome(service=Service(executable_path=url_chrome), options=chrome_options)
driver.maximize_window()

LPB = driver.get('https://lpbank.com.vn/lai-suat-2/')
LPB_soup= BeautifulSoup(driver.page_source, 'html')

MPB_value_mass = LPB_soup.find_all('table')[1]
LPB_table_OFF_mass =  [MPB_value_mass.find_all('tr')[y].find_all('td')[1].text.strip() for y in [5,6,7,10,16,17,20,21,23]]

LPB_value_af = LPB_soup.find_all('table')[2]
LPB_table_OFF_af = [LPB_value_af.find_all('tr')[y].find_all('td')[1].text.strip() for y in [2,3,4,7,13,14,17,18,20]]

LPB_value_onl_mass = LPB_soup.find_all('table')[3]
LPB_table_ONL_mass = [LPB_value_onl_mass.find_all('tr')[y].find_all('td')[1].text.strip() for y in [5,6,7,10,16,17,20,21,23]]

driver.close()

In [36]:
Update_inst.loc[5, ['MMYYYY',
                    'Bank_code', 
                    'Volume_type',
                    "Off_1M","Off_2M","Off_3M", "Off_6M", "Off_12M","Off_13M","Off_18M", "Off_24M", "Off_36M",
                    "Onl_1M","Onl_2M","Onl_3M","Onl_6M", "Onl_12M","Onl_13M","Onl_18M", "Onl_24M", "Onl_36M"]]\
                 = [datetime.datetime.now().strftime("%d/%m/%Y"),
                    'LVB',
                    'Mass',
                    *LPB_table_OFF_mass,
                    *LPB_table_ONL_mass
                    ]
Update_inst.loc[6, ['MMYYYY',
                    'Bank_code', 
                    'Volume_type',
                    "Off_1M","Off_2M","Off_3M", "Off_6M", "Off_12M","Off_13M","Off_18M", "Off_24M", "Off_36M",
                    "Onl_1M","Onl_2M","Onl_3M","Onl_6M", "Onl_12M","Onl_13M","Onl_18M", "Onl_24M", "Onl_36M"]]\
                 = [datetime.datetime.now().strftime("%d/%m/%Y"),
                    'LVB',
                    'UpperMass',
                    *LPB_table_OFF_mass,
                    *LPB_table_ONL_mass
                    ]
Update_inst.loc[7, ['MMYYYY',
                    'Bank_code', 
                    'Volume_type',
                    "Off_1M","Off_2M","Off_3M", "Off_6M", "Off_12M","Off_13M","Off_18M", "Off_24M", "Off_36M",
                    "Onl_1M","Onl_2M","Onl_3M","Onl_6M", "Onl_12M","Onl_13M","Onl_18M", "Onl_24M", "Onl_36M"]]\
                 = [datetime.datetime.now().strftime("%d/%m/%Y"),
                    'LVB',
                    'UpperMass',
                    *LPB_table_OFF_af,
                    *LPB_table_ONL_mass
                 ]